---
  title: "JDBC: Introduction à Java Database Connectivity"
  description: "Introduction à la connection aux bases de données relationnelles en Java avec Introduction à Java Database Connectivity (JDBC)."
  categories: 
    - Java
    - I111
    - Lecture
    - JavaFX
    - IHM
---

 * JDBC est une API intégrée à Java qui permet un accès simple aux Systèmes de Gestion de Dases de Données Relationnelles (SGBDR). 
 * Son objectif est d'offrir une vision uniforme des SGDBR notament en s'appuyant sur SQL et en définissant des types adaptés à Java. 
 * C'est une API qui permet de se connecter à une base de données, d'évaluer des requêtes et de parcourir l'ensemble des résultats.
 * JDBC aussi adaptable pour utiliser les spécifités d'un SGBDR donné mais au prix de la portabilité.

## Système de gestion de base de données et Dataset

 * Dans le cadre de ce cours nous utiliserons le système de gestion de base données [h2](https://www.h2database.com). 
 * Il s'agit d'un SGDB écrit en Java qui peut s'executer comme un serveur indépendant ou depuis un programme Java. Il peut aussi être utilisé purement en mémoire ou avec persistence sur disque.

Pour l'utiliser il suffit de télécharger le fichier .jar de h2 par exemple la version [h2-2.2.224.jar](https://repo1.maven.org/maven2/com/h2database/h2/2.2.224/h2-2.2.224.jar) depuis un entrepôt maven.

Le serveur peut être exécuté simplement avec la commande suivante pour autoriser les connexions tcp et la création automatique d'une base de données lors du premier accès. 
**Dans ce notebook, ouvrir un terminal en cliquant sur + en haut à gauche pour ouvrir un terminal.**
H2 supporte aussi un mode de compatibilité avec POstgresQL que nous allons utiliser.

```bash
java -cp h2*.jar \
    org.h2.tools.Server \
        -webAllowOthers \
        -tcpAllowOthers \
        -pgAllowOthers \
        -ifNotExists 
```

Une fois le serveur lancé il est possible d'executer un Shell SQL dans un autre terminal. 
Ici en mode de compatibilité postgresql avec *user* comme login et *secret* comme password.

```bash
java -cp h2*.jar org.h2.tools.Shell \
  -url "jdbc:h2:tcp://localhost/~/testdb;MODE=PostgreSQL;DATABASE_TO_LOWER=TRUE" \
  -user user -password secret
```

In [1]:
%maven fish.payara.extras:payara-micro:6.2023.10
import fish.payara.micro.BootstrapException;
import fish.payara.micro.PayaraMicro;
import fish.payara.micro.PayaraMicroRuntime;

PayaraMicroRuntime instance = PayaraMicro.getInstance()
    .setUserLogFile("/tmp/payara.log")
    .bootStrap();

[2024-03-05T13:14:04.054+0000] [] [WARNING] [] [PayaraMicro] [tid: _ThreadID=34 _ThreadName=IJava-executor-0] [timeMillis: 1709644444054] [levelValue: 900] Payara Micro Runtime directory is located in a temporary file location which can be cleaned by system processes.



[2024-03-05T13:14:04.062+0000] [] [INFO] [] [PayaraMicro] [tid: _ThreadID=34 _ThreadName=IJava-executor-0] [timeMillis: 1709644444062] [levelValue: 800] Payara Micro Runtime directory is located at /tmp/payaramicro-6191761700226532509tmp



[2024-03-05T13:14:04.090+0000] [] [INFO] [] [fish.payara.micro.boot.runtime.PayaraMicroRuntimeBuilder] [tid: _ThreadID=34 _ThreadName=IJava-executor-0] [timeMillis: 1709644444090] [levelValue: 800] Built Payara Micro Runtime



[2024-03-05T13:14:05.299+0000] [] [INFO] [NCLS-CORE-00098] [javax.enterprise.system.core] [tid: _ThreadID=34 _ThreadName=IJava-executor-0] [timeMillis: 1709644445299] [levelValue: 800] Cannot find h2db client jar file, h2 jdbc driver will not be available by default.



[2024-03-05T13:14:05.559+0000] [] [INFO] [NCLS-CORE-00013] [javax.enterprise.system.core] [tid: _ThreadID=34 _ThreadName=IJava-executor-0] [timeMillis: 1709644445559] [levelValue: 800] Shutdown procedure finished



[2024-03-05T13:14:05.566+0000] [] [INFO] [NCLS-CORE-00098] [javax.enterprise.system.core] [tid: _ThreadID=34 _ThreadName=IJava-executor-0] [timeMillis: 1709644445566] [levelValue: 800] Cannot find h2db client jar file, h2 jdbc driver will not be available by default.



EvalException: A MultiException has 7 exceptions.  They are:
1. org.glassfish.hk2.api.UnsatisfiedDependencyException: There was no object available in __HK2_Generated_0 for injection at SystemInjecteeImpl(requiredType=AuthorizationService,parent=CommandSecurityChecker,qualifiers={},position=-1,optional=false,self=false,unqualified=null,8957317)
2. java.lang.IllegalArgumentException: While attempting to resolve the dependencies of com.sun.enterprise.admin.util.CommandSecurityChecker errors were found
3. java.lang.IllegalStateException: Unable to perform operation: resolve on com.sun.enterprise.admin.util.CommandSecurityChecker
4. java.lang.IllegalArgumentException: While attempting to resolve the dependencies of com.sun.enterprise.v3.admin.CommandRunnerImpl errors were found
5. java.lang.IllegalStateException: Unable to perform operation: resolve on com.sun.enterprise.v3.admin.CommandRunnerImpl
6. java.lang.IllegalArgumentException: While attempting to resolve the dependencies of com.su

In [2]:
import java.sql.SQLException;
import org.h2.tools.Server;

public class DBManager {
    public static void startDB() throws SQLException {
        Server.createTcpServer("-tcpPort", "9092", 
                               "-tcpAllowOthers","-pgAllowOthers",
                               "-ifNotExists","-baseDir","/tmp/db").start();
    }

    public static void stopDB() throws SQLException {
        Server.shutdownTcpServer("tcp://localhost:9092", "", true, true);
    }
}

CompilationException: 

In [3]:
//DBManager.stopDB();
DBManager.startDB();

CompilationException: 

Dans ce cours nous allons utiliser une base de données qui représente des données de transport (des bus) au format [GTFS](https://developers.google.com/transit/gtfs/reference/#general_transit_feed_specification_reference). Le chargement initial peut être un peu long. 

In [4]:
%%shell
java -cp h2*.jar org.h2.tools.RunScript \
     -url "jdbc:h2:tcp://localhost/Gtfs_RMTT;MODE=PostgreSQL;DATABASE_TO_LOWER=TRUE" \
     -user sa \
     -script data.sql

zsh:1: no matches found: h2*.jar


Définissons tout d'abord simplement une classe Java pour représenter l'une des entités de l'exemple, un arrêt de bus.
Nous utiliserons un record Java (non mutable).

In [5]:
record Stop(long id, String name, float lat, float lon, String code) { }

In [6]:
Stop stop=new Stop(1,"X",5.9333F, 43.1167F, "X");
stop

stop: Stop[id=1, name=X, lat=5.9333, lon=43.1167, code=X]

## The JDBC Driver for postgresql
Pour se connecter à un SGBDR spécifique il faut disposer d'une implantation particulière d'un Driver (qui devra être ajouté à l'application). 
Il existe quatre familles de drivers JDBC :

  * Type 1 - s'appuie sur un autre driver (par exemple JDBC-ODBC)
  * Type 2 - utilise une implatation native d'un driver coté client.
  * Type 3 - utilise un middleware pour convertir les appels JDBC en appel s propriétaires du SGBDR.
  * Type 4 - offre une connection directe depuis une implantation Java (thin driver).

Le type 4 est le plus courant. Il est indépendant de la plateforme et offre de meilleures performances en se connectant directement à la base de données. 

## Ouverture d'une connexion
Pour ouvrir une connection, il faut charger la classe de l'implantation du Driver et ouvrir une connexion. En théorie, on devrait utiliser ```Class.forName("nom de la classe d'implantation")``` en pratique JDBC le fait automatique en fonction de l'URL lors de la connexion.

La première chose à faire est d'ajouter l'implantation du Driver pour la  base de données visée. Par exemple, en ajoutant à Maven une dépendance pour PostgreSQL.

Pour ouvrir une connexion, on utilise une URL dont la forme est spécifique à chaque SGBD par exemple pour [MySQL](https://dev.mysql.com/doc/connector-j/5.1/en/connector-j-usagenotes-connect-drivermanager.html), [PostgreSQL](https://jdbc.postgresql.org/) ou [H2](http://www.h2database.com/html/features.html) (une base de données relationnel en pur Java):

  * ```jdbc:mysql://localhost:3306/myDatabase?user=username&password=password```
  * ```jdbc:postgresql://localhost/myDatabase```
  * ```jdbc:h2:mem:myDatabase```

Les informations d'authentification peuvent parfois être donnés dans l'URL ou lors de l'appel de la méthode ```Connection DriverManager.getConnection(jdbcURL)```.

In [7]:
import java.sql.Connection;
import java.sql.DriverManager;
//String jdbcURL="jdbc:postgresql://db:5432/dvdrental?user=postgres&password=changeme";
String jdbcURL="jdbc:h2:tcp://localhost/Gtfs_RMTT;MODE=PostgreSQL;DATABASE_TO_LOWER=TRUE";

Connection connection = DriverManager.getConnection(jdbcURL,"sa","")


EvalException: No suitable driver found for jdbc:h2:tcp://localhost/Gtfs_RMTT;MODE=PostgreSQL;DATABASE_TO_LOWER=TRUE

## Exécution de requêtes SQL

Pour exécuter des requetes SQL on utilise des instances de la classes ```Statement```.

Il existe trois types de statements :

  * statement: requêtes simples
  * prepared statement: requêtes précompilées
  * callable statement: procédures stockées

Un statement est créé à partir d'une connection.

## Les requêtes simples

La méthode à appeler est différente suivant la nature de la requêtes SQL que l’on veut exécuter :

  * Consultation (select)
    * executeQuery() On parcours les t-uples avec un ResultSet

  * Mise à jour (update, insert, delete) ou gestion de la base de données (create table,...)
    * executeUpdate() renvoie le nombre de lignes modifiées

  * Type inconnu (ex. donné par un fonction sous forme de String) ou si la requêtes peut renvoyer plusieurs résultats (procédures stockées)
    * execute()

In [8]:
import java.sql.Statement;
import java.sql.ResultSet;
// Une requête simple
Statement statement = connection.createStatement();
String query1 = "SELECT * FROM gtfs_stops LIMIT 5";
ResultSet resultSet = statement.executeQuery(query1);

EvalException: Cannot invoke "java.sql.Connection.createStatement()" because "REPL.$JShell$76.connection" is null

## Parcours des résultats
* executeQuery() retourne de ResultSet
* L’interface ResultSet définit les méthodes pour accéder au valeur des attributs
  * getXXX(int numéroDeColonne)
  * getXXX(String nomDeColonne)
  * XXX désigne le type Java de la valeur que l'on va récupérer (Byte, Boolean, AsciiStream, Short, String UnicodeStream, Int Bytes, BinaryStream, Long, Date, Object, Float, Time, BigDecimal,TimeStamp)
* A Noter : données volumineuses (ex. Blob)
  * Ouverture d'un flux

In [9]:
import java.util.List;
import java.util.ArrayList;
List<Stop> stops = new ArrayList<>();
while (resultSet.next()) {
        stops.add(new Stop(resultSet.getLong("stop_id"),
                resultSet.getString("stop_name"),
                resultSet.getFloat("stop_lat"),
                resultSet.getFloat("stop_lon"),
                resultSet.getString("stop_code")));
}
System.out.println(stops);

CompilationException: 

In [10]:
Statement statement = connection.createStatement();
String query1 = """
    INSERT INTO gtfs_agency(agency_id, agency_name, agency_url, agency_timezone, agency_lang)
    VALUES(2, 'Neverland', 'http://nowhere.fr', 'Europe/Paris','fr')""";
int numberOfChanges = statement.executeUpdate(query1);

EvalException: Cannot invoke "java.sql.Connection.createStatement()" because "REPL.$JShell$76.connection" is null

## Les exceptions
* Erreur dans le code SQL : SQLException
* Avertissement lors de l'exécution (SQLWarning)
  * Problèmes de conversion de données (DataTruncation - sous-classe de SQLWarning)

In [11]:
//Traitement d'une erreur SQL
import java.sql.SQLException;
String wrongQuery = " SELECT * FROM Employee" ;
try (Connection connection = DriverManager.getConnection(jdbcURL,"sa","");
            Statement statement = connection.createStatement();
            ResultSet resultSet = statement.executeQuery(wrongQuery)) {
        // Do stuff here
        } catch (SQLException e) {
            //Erreur lors de la requête
            System.out.println(e.getMessage());        
        }

No suitable driver found for jdbc:h2:tcp://localhost/Gtfs_RMTT;MODE=PostgreSQL;DATABASE_TO_LOWER=TRUE


## Un exemple complet

Voilà un exemple complet avec la classique des exceptions. Attention, quand une connection n'est plus utilisée il faut la fermer pour libérer les ressources sur le SGBD.

In [12]:
 String jdbcURL="jdbc:h2:tcp://localhost/Gtfs_RMTT;MODE=PostgreSQL;DATABASE_TO_LOWER=TRUE";

        Connection connection = null;
        ResultSet resultSet = null;
        Statement statement = null;
        try {
            connection = DriverManager.getConnection(jdbcURL,"sa","");
            statement = connection.createStatement();
            String query1 = "SELECT * FROM gtfs_stops LIMIT 5";
            resultSet = statement.executeQuery(query1);

            List<Stop> stops = new ArrayList<>();
            while (resultSet.next()) {
             stops.add(new Stop(resultSet.getLong("stop_id"),
                resultSet.getString("stop_name"),
                resultSet.getFloat("stop_lat"),
                resultSet.getFloat("stop_lon"),
                resultSet.getString("stop_code")));
         }
            System.out.println(stops);
        } catch (SQLException e) {
            //Erreur lors de la requête
            e.printStackTrace();
        } finally {
            if (resultSet != null) {
                try {
                    resultSet.close();
                } catch (SQLException e) { /* ignored */}
            }
            if (connection != null) {
                try {
                    connection.close();
                } catch (SQLException e) { /* ignored */}
            }
        }

java.sql.SQLException: No suitable driver found for jdbc:h2:tcp://localhost/Gtfs_RMTT;MODE=PostgreSQL;DATABASE_TO_LOWER=TRUE


	at java.sql/java.sql.DriverManager.getConnection(DriverManager.java:708)


	at java.sql/java.sql.DriverManager.getConnection(DriverManager.java:230)


	at REPL.$JShell$87.do_it$($JShell$87.java:59)


	at java.base/jdk.internal.reflect.DirectMethodHandleAccessor.invoke(DirectMethodHandleAccessor.java:103)


...

	at io.github.spencerpark.ijava.execution.IJavaExecutionControl.lambda$execute$1(IJavaExecutionControl.java:95)


	at java.base/java.util.concurrent.FutureTask.run(FutureTask.java:317)


	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1144)


	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:642)


	at java.base/java.lang.Thread.run(Thread.java:1583)


A partir de Java 7 une syntaxe particulière (try-with-ressources) est possible pour fermer automatiquement certaines ressources dont les connections et les ResultSet JDBC.

In [13]:
 String jdbcURL="jdbc:h2:tcp://localhost/Gtfs_RMTT;MODE=PostgreSQL;DATABASE_TO_LOWER=TRUE";

        String query1 = "SELECT * FROM gtfs_stops LIMIT 5";
        List<Stop> stops = new ArrayList<>();

        try (Connection connection = DriverManager.getConnection(jdbcURL,"sa","")) {
            ResultSet resultSet = null;
            Statement statement = null;
            statement = connection.createStatement();
            resultSet = statement.executeQuery(query1);

            while (resultSet.next()) {
             stops.add(new Stop(resultSet.getLong("stop_id"),
                resultSet.getString("stop_name"),
                resultSet.getFloat("stop_lat"),
                resultSet.getFloat("stop_lon"),
                resultSet.getString("stop_code")));
            }
            System.out.println(stops);
        } catch (SQLException e) {
            //Erreur lors de la requête
            e.printStackTrace();
        }

java.sql.SQLException: No suitable driver found for jdbc:h2:tcp://localhost/Gtfs_RMTT;MODE=PostgreSQL;DATABASE_TO_LOWER=TRUE


	at java.sql/java.sql.DriverManager.getConnection(DriverManager.java:708)


	at java.sql/java.sql.DriverManager.getConnection(DriverManager.java:230)


	at REPL.$JShell$89.do_it$($JShell$89.java:59)


	at java.base/jdk.internal.reflect.DirectMethodHandleAccessor.invoke(DirectMethodHandleAccessor.java:103)


...

	at io.github.spencerpark.ijava.execution.IJavaExecutionControl.lambda$execute$1(IJavaExecutionControl.java:95)


	at java.base/java.util.concurrent.FutureTask.run(FutureTask.java:317)


	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1144)


	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:642)


	at java.base/java.lang.Thread.run(Thread.java:1583)


## Types  Java/JDBC et SQL
* Malgré SQL les SGBD présentent des différences de types
* JDBC masque ces différences en définissant ses propres types SQL (constantes de la classe Types)
* Le driver assure la conversion
  * SQL vers Java lors de la lecture
  * Java vers SQL lors du passage de paramètres
* Utilisés explicitement avec les methodes getXXX() (et setXXX())
* Parfois plusieurs choix (presque tous les types SQL peuvent être retrouvés par getString())
  * CHAR et VARCHAR : getString()
  * LONGVARCHAR : getAsciiStream() et getCharacterStream()
  * BINARY et VARBINARY : getBytes()
  * LONGVARBINARY : getBinaryStream()
  * REAL : getFloat(), DOUBLE et FLOAT : getDouble()
  * DECIMAL et NUMERIC : getBigDecimal()
  * DATE : getDate(), TIME : getTime(), TIMESTAMP :getTimestamp()

* Par défaut une connexion est ouverte en « auto-
commit » :
  * un commit est automatiquement lancé après chaque requete SQL qui fait une mise à jour
* Pour un contrôle plus fin on utilise
```java
conn.setAutoCommit(false) //pour le désactiver
conn.commit() //pour valider la transaction
conn.rollback() //pour annuler la transaction
```

## Précompilation des requêtes
* Si les requêtes fabriquées à partir de String changent (paramètres) :
  * Elles sont compilées à chaque appel d'où une perte de
performances
* JDBC permet de ne compiler la requête qu'une fois (si le
SGBD le supporte)
  * En indiquant les paramètres de façon générique
  * En fixant leur valeur (sans changer la requête) au moment
de l'exécution
* Deux Statement particuliers :
  * Les requêtes paramétrées (PreparedStatement)
Les procédures stockées (CallableStatement)

In [14]:
import java.sql.PreparedStatement;
List<Stop> stops = new ArrayList<>();
try (Connection connection = DriverManager.getConnection(jdbcURL,"sa","")) {
 PreparedStatement preparedStatement = 
     connection.prepareStatement("SELECT * FROM gtfs_stops WHERE stop_id = ?");
 int[] ids={100034,100040,100063};
 //Juste pour illuster le cas ou une requête revient fréquement, mieux vaudrait une seule requêtes SQL
 for(int id:ids) {
    preparedStatement.setInt(1,id);
    ResultSet resultSet = preparedStatement.executeQuery();
            while (resultSet.next()) {
             stops.add(new Stop(resultSet.getLong("stop_id"),
                resultSet.getString("stop_name"),
                resultSet.getFloat("stop_lat"),
                resultSet.getFloat("stop_lon"),
                resultSet.getString("stop_code")));
            }
 }
 } catch (SQLException e) {
            //Erreur lors de la requête
            e.printStackTrace();
        }
System.out.println(stops);

java.sql.SQLException: No suitable driver found for jdbc:h2:tcp://localhost/Gtfs_RMTT;MODE=PostgreSQL;DATABASE_TO_LOWER=TRUE


	at java.sql/java.sql.DriverManager.getConnection(DriverManager.java:708)


	at java.sql/java.sql.DriverManager.getConnection(DriverManager.java:230)


	at REPL.$JShell$91.do_it$($JShell$91.java:58)


	at java.base/jdk.internal.reflect.DirectMethodHandleAccessor.invoke(DirectMethodHandleAccessor.java:103)


...

	at java.base/java.util.concurrent.FutureTask.run(FutureTask.java:317)


	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1144)


	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:642)


	at java.base/java.lang.Thread.run(Thread.java:1583)


[]


## Les transactions

In [15]:
Connection connection = DriverManager.getConnection(jdbcURL, "sa","");


//Create a account table
String createAccountTableSql="CREATE TABLE account ("+
    "id SERIAL PRIMARY KEY,"+
    "name VARCHAR(100) NOT NULL,"+
    "balance DEC(15,2) NOT NULL)";
Statement statement = connection.createStatement();
statement.executeUpdate(createAccountTableSql);
statement.close();

    
//Create accounts for Alice and Bob with a prepared statement
String createAccountSql = "INSERT INTO account(name,balance) VALUES(?,?);";
PreparedStatement pstmt = connection.prepareStatement(createAccountSql);
pstmt.setString(1,"Bob");
pstmt.setInt(2, 1000);
pstmt.executeUpdate();

pstmt.setString(1,"Alice");
pstmt.setInt(2, 1000);
pstmt.executeUpdate();

pstmt.close();

PreparedStatement pstmtIncreaseAccount = connection.prepareStatement("UPDATE account SET balance = balance + ? WHERE id = ?");

//save autoCommit state
boolean autoCommit = connection.getAutoCommit();
try {
    connection.setAutoCommit(false);
    //remove 500€ to Bob
    pstmtIncreaseAccount.setInt(1,-500);
    pstmtIncreaseAccount.setInt(2,1);
    pstmtIncreaseAccount.executeUpdate();
    
    //add 500€ to Alice
    pstmtIncreaseAccount.setInt(1,500);
    pstmtIncreaseAccount.setInt(2,2);
    pstmtIncreaseAccount.executeUpdate();
    connection.commit();
} catch (SQLException exc) {
    //Cancel the whole transaction if there is a problem.
    connection.rollback();
} finally {
    //restore autoCommit state
    connection.setAutoCommit(autoCommit);
}

//Drop the table
connection.createStatement().executeUpdate("DROP TABLE account");

connection.close();

EvalException: No suitable driver found for jdbc:h2:tcp://localhost/Gtfs_RMTT;MODE=PostgreSQL;DATABASE_TO_LOWER=TRUE

## Procédure stockées
CallableStatement permet d'appeller une procédure stockée directement sur le SGBD. 
TODO

## Metadata
* JDBC permet de récupérer des informations sur le type de données que l'on vient de récupérer par un SELECT (interface ResultSetMetaData),
* mais aussi sur la base de données elle-même (interface DatabaseMetaData)
* Les données que l'on peut récupérer avec DatabaseMetaData dépendent du SGBD avec lequel on travaille

In [17]:
//Lecture des metadonnées du serveur de base de données
import java.sql.*;

String jdbcURL="jdbc:h2:tcp://localhost/Gtfs_RMTT;MODE=PostgreSQL;DATABASE_TO_LOWER=TRUE";

try (Connection connection = DriverManager.getConnection(jdbcURL,"sa","")) {

                DatabaseMetaData metadata = connection.getMetaData();

                //Print info about the database system
                System.out.println("Database: " + metadata.getDatabaseProductName()
                        + " " + metadata.getDatabaseMajorVersion() + "." + metadata.getDatabaseMinorVersion());

                System.out.println(connection.getCatalog() + " " + connection.getSchema());

                //Retrieving the list of database names
                ResultSet tables = metadata.getTables(connection.getCatalog(),
                        connection.getSchema(),
                        "SIMPLEJDBC_PERSON", null);

                if (tables.next()) {
                    System.out.println("Table " + tables.getString("TABLE_NAME") + " already exist.");
                } else {
                    connection.createStatement().execute("CREATE TABLE \"SIMPLEJDBC_PERSON\"(" +
                            "    id INT PRIMARY KEY NOT NULL, " +
                            "    firstname VARCHAR(100))");
                    System.out.println("Table SIMPLEJDBC_PERSON created.");
                }
            } catch (SQLException e) {
                e.printStackTrace();
            }


java.sql.SQLException: No suitable driver found for jdbc:h2:tcp://localhost/Gtfs_RMTT;MODE=PostgreSQL;DATABASE_TO_LOWER=TRUE


	at java.sql/java.sql.DriverManager.getConnection(DriverManager.java:708)


	at java.sql/java.sql.DriverManager.getConnection(DriverManager.java:230)


	at REPL.$JShell$93.do_it$($JShell$93.java:59)


	at java.base/jdk.internal.reflect.DirectMethodHandleAccessor.invoke(DirectMethodHandleAccessor.java:103)


...

	at io.github.spencerpark.ijava.execution.IJavaExecutionControl.lambda$execute$1(IJavaExecutionControl.java:95)


	at java.base/java.util.concurrent.FutureTask.run(FutureTask.java:317)


	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1144)


	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:642)


	at java.base/java.lang.Thread.run(Thread.java:1583)


In [18]:
//Traitement des metadonnées d'un ResulSet
//pour la découverte du nombre, du type et du nom des colonnes. 
String jdbcURL="jdbc:h2:tcp://localhost/Gtfs_RMTT;MODE=PostgreSQL;DATABASE_TO_LOWER=TRUE";

try (Connection connection = DriverManager.getConnection(jdbcURL,"sa","");     
     ResultSet rs = connection.createStatement().executeQuery("SELECT * FROM gtfs_stops")) {

  ResultSetMetaData rsmd = rs.getMetaData();
  int nbColonnes = rsmd.getColumnCount();
  for (int i = 1; i <= nbColonnes; i++) {
    String typeColonne = rsmd.getColumnTypeName(i);
    String nomColonne = rsmd.getColumnName(i);
    System.out.println("Colonne " + i + " de nom " + nomColonne + " de type " + typeColonne);
  }
}

EvalException: No suitable driver found for jdbc:h2:tcp://localhost/Gtfs_RMTT;MODE=PostgreSQL;DATABASE_TO_LOWER=TRUE

## ResultSet modifiables
Il est aussi possible de paramétrer un ResultSet pour qu'il soit modifiable pendant la consultation et/ou que l'on puisse revenir en arrière.

In [19]:
try (Connection connection = DriverManager.getConnection(jdbcURL,"sa","");     
     ResultSet rs = connection.createStatement(ResultSet.TYPE_SCROLL_INSENSITIVE,ResultSet.CONCUR_UPDATABLE)
         .executeQuery("SELECT * FROM gtfs_stops")) {
  
//to be complete    

}

EvalException: No suitable driver found for jdbc:h2:tcp://localhost/Gtfs_RMTT;MODE=PostgreSQL;DATABASE_TO_LOWER=TRUE

## Datasources
Il est important de gérer l'ensemble des connexions qui sont ouvertes et idéalement de les réutiliser plutôt que de fermer/ouvrir. Cela peut être fait "à la main" en utilisant le concept de DataSource ou en utilisant une librairie comme [Apache DBCP](http://commons.apache.org/proper/commons-dbcp/).

L'utilisation est simple on définit une classe qui paramètre une datasource qui gère un pool de connexion et fournit des connections : 

In [20]:
%%loadFromPOM
<dependency>
    <groupId>org.apache.commons</groupId>
    <artifactId>commons-dbcp2</artifactId>
    <version>2.7.0</version>
</dependency>

In [21]:
import org.apache.commons.dbcp2.BasicDataSource;

import java.sql.Connection;
import java.sql.SQLException;

public class DBCPDataSource {

    private static BasicDataSource ds = new BasicDataSource();

    static {
        ds.setUrl(jdbcURL);
        ds.setUsername("sa");
        ds.setPassword("");

        ds.setMinIdle(5);
        ds.setMaxIdle(10);
        ds.setMaxOpenPreparedStatements(100);
    }

    private DBCPDataSource() {
    }

    public static Connection getConnection() throws SQLException {
        return ds.getConnection();
    }
}

Il suffit ensuite de demander et de rendre une connexion (le close ne ferme pas forcément la connexion).

In [22]:
try (Connection connection=DBCPDataSource.getConnection()) {
    Statement statement = connection.createStatement();
    ResultSet resultSet = statement.executeQuery("SELECT * FROM gtfs_stops LIMIT 2");
    List<Stop> stops = new ArrayList<>();
    
            while (resultSet.next()) {
             stops.add(new Stop(resultSet.getLong("stop_id"),
                resultSet.getString("stop_name"),
                resultSet.getFloat("stop_lat"),
                resultSet.getFloat("stop_lon"),
                resultSet.getString("stop_code")));
            }
            System.out.println(stops);
        } catch (SQLException e) {
            //Erreur lors de la requête
            e.printStackTrace();
        }

Cannot create JDBC driver of class '' for connect URL 'jdbc:h2:tcp://localhost/Gtfs_RMTT;MODE=PostgreSQL;DATABASE_TO_LOWER=TRUE'
java.sql.SQLException: No suitable driver
	at java.sql/java.sql.DriverManager.getDriver(DriverManager.java:300)
	at org.apache.commons.dbcp2.DriverFactory.createDriver(DriverFactory.java:60)
	at org.apache.commons.dbcp2.BasicDataSource.createConnectionFactory(BasicDataSource.java:472)
	at org.apache.commons.dbcp2.BasicDataSource.createDataSource(BasicDataSource.java:538)
	at org.a

pache.commons.dbcp2.BasicDataSource.getConnection(BasicDataSource.java:753)
	at REPL.$JShell$100$DBCPDataSource.getConnection($JShell$100.java:79)
	at REPL.$JShell$101.do_it$($JShell$101.java:59)
	at java.base/jdk.internal.reflect.DirectMethodHandleAccessor.invoke(DirectMethodHandleAccessor.java:103)
	at java.base/java.lang.reflect.Method.invoke(Method.java:580)
	at io.github.spencerpark.ijava.execution.IJavaExecutionControl.lambda$execute$1(IJavaExecutionControl.java:95)
	at java.base/java.util.concurrent.

java.sql.SQLException: Cannot create JDBC driver of class '' for connect URL 'jdbc:h2:tcp://localhost/Gtfs_RMTT;MODE=PostgreSQL;DATABASE_TO_LOWER=TRUE'


	at org.apache.commons.dbcp2.DriverFactory.createDriver(DriverFactory.java:75)


	at org.apache.commons.dbcp2.BasicDataSource.createConnectionFactory(BasicDataSource.java:472)


...

	at java.base/java.lang.Thread.run(Thread.java:1583)


Caused by: java.sql.SQLException: No suitable driver


	at java.sql/java.sql.DriverManager.getDriver(DriverManager.java:300)


	at org.apache.commons.dbcp2.DriverFactory.createDriver(DriverFactory.java:60)


	... 12 more


## Batch Update
Pour des ajouts en masse pensez à utiliser les batch updates mais attention aux transactions.

In [23]:
try (Connection connection = DriverManager.getConnection(jdbcURL,"sa","")) {  
     
  //Création d'une table PERSONNE
  String createAccountTableSql="CREATE TABLE PERSONNE ("+
    "id SERIAL PRIMARY KEY,"+
    "email VARCHAR(100) NOT NULL,"+
    "nom VARCHAR(100) NOT NULL,"+
    "prenom VARCHAR(100) NOT NULL)";
  Statement statement = connection.createStatement();
  statement.executeUpdate(createAccountTableSql);
  statement.close();     
     
  PreparedStatement pstmt = 
        connection.prepareStatement("INSERT INTO PERSONNE (email, nom, prenom) VALUES(?, ?, ?)");
 
  //On ajoute les exécution une à une ...
  connection.setAutoCommit(false);
 
  // On fixe les paramètres de la première requête à exécuter
  pstmt.setString( 1, "p1.n1@...");
  pstmt.setString( 2, "n1" );
  pstmt.setString( 3, "p1" );
  // Et on l'ajoute au batch
  pstmt.addBatch();
 
  // On fixe les paramètres de la seconde requête à exécuter
  pstmt.setString( 1, "p2.n2@...");
  pstmt.setString( 2, "n2" );
  pstmt.setString( 3, "p2" );
  // Et on l'ajoute au batch
  pstmt.addBatch();
 
  // On ajoute autant de requêtes que nécessaire
  //Par exemple en lisant un flux de données.


  //On créée un tableau d'entiers pour recevoir les résultats.
  //et on execute toutes les mises à jour en une fois.
  int[] affectedRecords = pstmt.executeBatch();
 
  System.out.println(Arrays.toString(affectedRecords));
    
  //On valide les changements.
  connection.commit();
    
  //Drop the table
  connection.createStatement().executeUpdate("DROP TABLE PERSONNE");    
}    

EvalException: No suitable driver found for jdbc:h2:tcp://localhost/Gtfs_RMTT;MODE=PostgreSQL;DATABASE_TO_LOWER=TRUE

## DAO
La couche DAO (Data abstraction Object) permet d'abstraire totalement l'interaction avec le support de persistance (ici la base de donnée relationnelle).
Elle centralise les opérations spécifique et transforme les exceptions spécifiques en exceptions applicatives.

Une classe DAO peut être associée à chaque entité. Comme la structure est très répétive, la définition d'une interface et d'un classe générique est fortement recommandée. 

Ci-dessous les grandes lignes d'une DAO de base.

In [24]:
//Définir plus finement les exceptions
public class DataAccessException extends Exception {}

//L'utilisation de la pagination des résultats est obligatoire
public record Page<T>(int pageNumber, int pageSize, List<T> resultList) {}

//Remplacer par une Exception métier
public interface DAO<E> {
    E find(long id) throws DataAccessException;    
    Page<E> findAll() throws DataAccessException;
    Page<E> findAll(int pageNumber, int pageSize) throws DataAccessException; 
    E persist(E entity) throws DataAccessException;
    E update(E entity) throws DataAccessException;
    void remove(E entity) throws DataAccessException;
    void close() throws DataAccessException;
}


//Un record est une type de class particulier introduit avec Java 16
//qui permet de déclarer simplement des classes dédiées au transfert de données 
//cf. https://docs.oracle.com/en/java/javase/16/language/records.html
public record Personne(long id,String email,String nom,String prenom) { }

public class PersonDAO implements DAO<Personne> {
    //La connection vers la base de données 
    private final Connection connection;
    private final PreparedStatement pstmt_insert;
    private final PreparedStatement pstmt_find;

    /*
    // Un construction qui obtient automatiquement la connection
    public PersonDAO() throws DataAccessException {
        this(METTRE ICI LE CODE D'OBTENTION D'UNE CONNECTION DANS LA DATASOURCE);
    }*/
    
    //ATTENTION A LA GESTION DES TRANSACTIONS
    public PersonDAO(Connection connection) throws DataAccessException {
        try {
            this.connection = connection; 
            pstmt_insert = connection.prepareStatement("INSERT INTO PERSONNE (email, nom, prenom) VALUES(?, ?, ?)");
            pstmt_find = connection.prepareStatement("SELECT id, email, nom, prenom FROM PERSONNE WHERE id = ?");
        } catch (java.sql.SQLException e) {throw new DataAccessException();}
    }
    
    public Personne find(long id) {
        return new Personne(-1,"a.b@x.y.fr","MonNom","MonPrenom");
    }

    //Définir une classe générique pour la pagination
    public Page<Personne> findAll() {return findAll(1,100);}
    public Page<Personne> findAll(int pageNumber, int pageSize) { 
        List<Personne> resultList = new ArrayList<Personne>();
        //Completer avec un prepared statement 
        return new Page<Personne>(pageNumber, pageSize, resultList); 
    }

    public Personne persist(Personne personne) { 
        return personne; 
    }
    public Personne update(Personne personne) {
        //Completer avec un prepared statement 
        return personne; 
    }
    public void remove(Personne personne) {
        //Completer avec un prepared statement 
        }
    public void close() throws DataAccessException {
        try {
            connection.close(); 
        } catch (java.sql.SQLException e) {throw new DataAccessException();}
    }
}    

## Un exemple "Simple"

Pour finir voilà un exemple "Simple" d'utilisation d'une Datasource et de d'une DAO avec JDBC avec le SGBD H2.

L'exemple ci-dessous utilise une base de donnée H2 qui est exécutée directement depuis l'application.

In [25]:
%%shell
mkdir -p /home/jovyan/work/src/github/ebpro
cd /home/jovyan/work/src/github/ebpro
rm -rf sample-jdbcdao
git clone https://github.com/ebpro/sample-jdbcdao

Cloning into 'sample-jdbcdao'...


La classe StartH2 est un exemple pour exécuter H2 depuis Java. A ne faire qu'une seule fois dans le notebook.

In [26]:
%%shell
cd /home/jovyan/work/src/github/ebpro/sample-jdbcdao
rm -f /tmp/db/test.mv.db
mvn --quiet -ntp package exec:java -Dexec.mainClass=fr.univtln.bruno.jdbcdao.persistence.StartH2 &
sleep 2

In [27]:
%%shell
cd /home/jovyan/work/src/github/ebpro/sample-jdbcdao
mvn --quiet package 

Mar 05, 2024 1:14:13 PM fr.univtln.bruno.jdbcdao.persistence.daos.AbstractDAO <init>


Mar 05, 2024 1:14:13 PM fr.univtln.bruno.jdbcdao.persistence.daos.AbstractDAO <init>


Mar 05, 2024 1:14:13 PM fr.univtln.bruno.jdbcdao.persistence.daos.AbstractDAO <init>


Mar 05, 2024 1:14:13 PM fr.univtln.bruno.jdbcdao.persistence.daos.AbstractDAO <init>


[main] INFO fr.univtln.bruno.jdbcdao.persistence.daos.PersonneDAOTest - Personne(id=351, nom=X, prenom=Y)


...

Mar 05, 2024 1:14:14 PM fr.univtln.bruno.jdbcdao.persistence.StartH2 main


INFO: Server start error. Exception opening port "9092" (port may be in use), cause: "java.net.BindException: Address already in use" [90061-224]


[main] INFO fr.univtln.bruno.jdbcdao.persistence.daos.PersonneDAOTest - Personne(id=351, nom=X, prenom=Y)


Mar 05, 2024 1:14:14 PM fr.univtln.bruno.jdbcdao.persistence.daos.AbstractDAO <init>


Mar 05, 2024 1:14:14 PM fr.univtln.bruno.jdbcdao.persistence.daos.AbstractDAO <init>


In [28]:
%%shell
cd /home/jovyan/work/src/github/ebpro/sample-jdbcdao
mvn exec:java -Dexec.mainClass=fr.univtln.bruno.jdbcdao.persistence.App

[INFO] Scanning for projects...


[INFO] 


[INFO] ----------------------< fr.univtln.bruno:JdbcDao >----------------------


[INFO] Building JDBC DAO 1.0-SNAPSHOT


[INFO]   from pom.xml


...

[INFO] BUILD SUCCESS


[INFO] ------------------------------------------------------------------------


[INFO] Total time:  0.574 s


[INFO] Finished at: 2024-03-05T13:14:16Z


[INFO] ------------------------------------------------------------------------


Cette application peut être étudiée, testée et modifiée depuis l'éditeur Visual Code intégré (cliquer sur + en haut à droite puis lancer VS Code IDE).